In [1]:
import requests
import pandas as pd
import time
import numpy as np
import ast
from datetime import (datetime, timedelta, date)



import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import (MarketOrderRequest, TrailingStopLossDetails)

from ta import trend
from ta import momentum

from oanda_order_creation import *

In [2]:
def convert_to_json(text):
    try:
        return ast.literal_eval(text)
    except:
#         print(text)
        return {}

In [3]:
def clean_candle_data(df, price_type = 'ask' ):
    
    df_modified = df[['complete', 'currency_pair', 'time', 'volume', price_type]]
    df_modified[price_type] = df_modified[price_type].astype(str)
    df_modified[price_type] = df_modified[price_type].apply(convert_to_json)
    
    price_df = pd.json_normalize(df_modified[price_type])
    
    df_modified = pd.merge(df_modified, price_df,
                          left_index=True, right_index=True)
    
    df_modified = df_modified[['complete','currency_pair', 'time', 'volume','o', 'h', 'l', 'c']]
    df_modified.columns = ['complete','currency_pair', 'time', 'volume','price_open','price_high', 'price_low', 'price_close']
    
    df_modified[["price_open", "price_high", "price_low", "price_close"]] = df_modified[["price_open", "price_high",
                                                                                         "price_low", "price_close"]].apply(pd.to_numeric)
    df_modified["time"] = pd.to_datetime(df_modified["time"])
    
    return df_modified

In [4]:
def get_macd_indicator(df, window_slow: int = 26, window_fast: int = 12, window_sign: int = 9, fillna: bool = False):
    # Innitialising MACD indicator
    indicator_macd = trend.MACD(close = df["price_close"])
    # Create columns for MACCD line, singal line and macd histogram (macd crosses signal line)
    df['macd'] = indicator_macd.macd()
    df['macd_signal'] = indicator_macd.macd_signal()
    df['macd_signal_diff'] = indicator_macd.macd_diff()
    return df

In [12]:
account_id = "101-003-18483320-003"
access_token = "87a216bb7b0e640e4af97a43499b58b9-4803472d0b66dd35b5d7541f3105caf5"

#### GET X NUMBER OF CANDLES FOR A SPECIFIC INTSTRUMENTS (REAL TIME FOR LAST CANDLE)

In [ ]:
def get_candle_data(instrument, granularity, number_of_candles=200):
    client = oandapyV20.API(access_token=access_token, environment = "practice" or "live")
    params = {
      "count": number_of_candles,
      "granularity": granularity
    }
    r = instruments.InstrumentsCandles(instrument=instrument,
                                   params=params)
    response = client.request(r)
    df = pd.DataFrame(response["candles"])
    df["currency_pair"] = instrument
    return clean_candle_data(df, "mid")
    return df
    

In [ ]:
extraction = False
while extraction == False:
    time.sleep(0.5)
    time_now = datetime.utcnow()
    print(time_now)
    if (time_now.minute % 5 == 0) & (time_now.second == 0):
#     if (time_now.second % 5 == 0):
        extraction = True

    

while extraction == True:    
    df = get_candle_data("EUR_USD", "M5", 2000)

    df_with_indicators =  get_macd_indicator(df)

    indicator_sma = trend.SMAIndicator(close = df_with_indicators["price_close"], window=200)
    df_with_indicators['sma_200'] = indicator_sma.sma_indicator()
    
    indicator_rsi = momentum.RSIIndicator(close = df_with_indicators["price_close"], window=14)
    df_with_indicators['rsi_14'] = indicator_rsi.rsi()

    df_with_macd_signals = df_with_indicators[df_with_indicators["complete"] == True]
    df_with_macd_signals = macd_buy_sell_signal(df_with_indicators)
    print(df_with_macd_signals["buy_sell_signal"].to_list()[-1])

    if df_with_macd_signals["buy_sell_signal"].to_list()[-1] == "buy":
        create_market_order("EUR_USD", "BUY", 10)
        print("Buy order created")
    elif df_with_macd_signals["buy_sell_signal"].to_list()[-1] == "sell":
        create_market_order("EUR_USD", "SELL", 10)
        print("Sell order created")
    print(df_with_macd_signals["macd_signal_diff"].to_list()[-1])
    time.sleep(300)
    




In [ ]:
extraction = False
while extraction == False:
#     time.sleep(0.5)
    time_now = datetime.utcnow()
    if (time_now.minute % 5 == 0) & (time_now.second == 0):
        print(time_now)
        extraction = True
time.sleep(5)
state = True
buy_count = 0
sell_count = 0
while state == True:
    df = get_candle_data("EUR_USD", "M5", 500)

    df_with_indicators =  get_macd_indicator(df)

    indicator_sma = trend.SMAIndicator(close = df_with_indicators["price_close"], window=200)
    df_with_indicators['sma_200'] = indicator_sma.sma_indicator()

    indicator_rsi = momentum.RSIIndicator(close = df_with_indicators["price_close"], window=14)
    df_with_indicators['rsi_14'] = indicator_rsi.rsi()

    df_with_indicators_complete = df_with_indicators[df_with_indicators["complete"] == True]

    if ((df_with_indicators_complete["macd_signal_diff"].iloc[-1] < 0) and
        (df_with_indicators_complete["macd_signal_diff"].iloc[-2] < 0) and
        (df_with_indicators_complete["macd_signal_diff"].iloc[-3] >= 0) and
        (df_with_indicators_complete["rsi_14"].iloc[-1] >= 45) and
        (df_with_indicators_complete["price_close"].iloc[-1] < df_with_indicators_complete["sma_200"].iloc[-1])):
        print(df_with_indicators_complete["time"].iloc[-1], "SELL")
        sell_count += 1
        print("sell_count" + str(sell_count))
        create_market_order_with_sl_tp("EUR_USD", "SELL", 500,
                                      round(float(df_with_indicators_complete["price_close"].iloc[-1]) + 10/10000,5),
                                      round(float(df_with_indicators_complete["price_close"].iloc[-1]) - 10/10000,5))

    elif ((df_with_indicators_complete["macd_signal_diff"].iloc[-1] > 0) and
         (df_with_indicators_complete["macd_signal_diff"].iloc[-2] > 0) and
         (df_with_indicators_complete["macd_signal_diff"].iloc[-3] <= 0) and
         (df_with_indicators_complete["rsi_14"].iloc[-1] <= 55 ) and
         (df_with_indicators_complete["price_close"].iloc[-1] > df_with_indicators_complete["sma_200"].iloc[-1])):
        print(df_with_indicators_complete["time"].iloc[-1], "BUY")
        buy_count += 1
        print("buy_count" + str(buy_count))
        create_market_order_with_sl_tp("EUR_USD", "BUY", 500,
                                      round(float(df_with_indicators_complete["price_close"].iloc[-1]) - 10/10000,5),
                                      round(float(df_with_indicators_complete["price_close"].iloc[-1]) + 10/10000,5))
    else:
        print(df_with_indicators_complete["time"].iloc[-1], "Nothing happens")
    time.sleep(300)

#### CREATE DIFFERENT ORDERS

In [ ]:
def create_market_order(instrument, order_type, order_quantity):
    order_quantity = order_quantity  if order_type == "BUY" else -abs(order_quantity)
    data = {
      "order": {
        "units": str(order_quantity),
        "instrument": instrument,
        "timeInForce": "FOK",
        "type": "MARKET",
        "positionFill": "DEFAULT"
      }
    }
    client = oandapyV20.API(access_token=access_token, environment = "practice" or "live")
    r = orders.OrderCreate(accountID = account_id, data=data)

    client.request(r)


In [ ]:
def create_market_order_with_sl_tp(instrument, order_type, order_quantity, sl_price, tp_price):
    order_quantity = order_quantity  if order_type == "BUY" else -abs(order_quantity)
    data = {
        "order": {
            "units": str(order_quantity),
            "instrument": instrument,
            "timeInForce": "FOK",
            "type": "MARKET",
            "positionFill": "DEFAULT",
            "stopLossOnFill": {
                "price": str(sl_price)
            },
            "takeProfitOnFill": {
                "price": str(tp_price)
            }
        },
    }
    client = oandapyV20.API(access_token=access_token, environment = "practice" or "live")
    r = orders.OrderCreate(accountID = account_id, data=data)

    client.request(r)


In [ ]:
def create_market_order_with_sl_only(instrument, order_type, order_quantity, sl_price):
    order_quantity = order_quantity  if order_type == "BUY" else -abs(order_quantity)
    data = {
        "order": {
            "units": str(order_quantity),
            "instrument": instrument,
            "timeInForce": "FOK",
            "type": "MARKET",
            "positionFill": "DEFAULT",
            "stopLossOnFill": {
                "price": str(sl_price)
            },

        },
    }
    client = oandapyV20.API(access_token=access_token, environment = "practice" or "live")
    r = orders.OrderCreate(accountID = account_id, data=data)

    client.request(r)


In [ ]:
def create_market_order_with_trailing_sl_tp(instrument, order_type, order_quantity, trailing_sl_pip, tp_price):
    order_quantity = order_quantity  if order_type == "BUY" else -abs(order_quantity)
    trailingStopLossOnFill = TrailingStopLossDetails(distance=trailing_sl_pip/10000)
    data = {
        "order": {
            "units": str(order_quantity),
            "instrument": instrument,
            "timeInForce": "FOK",
            "type": "MARKET",
            "positionFill": "DEFAULT",
            "trailingStopLossOnFill": trailingStopLossOnFill.data,
            "takeProfitOnFill": {
                "price": str(tp_price)
            }
        },
    }
    client = oandapyV20.API(access_token=access_token, environment = "practice" or "live")
    r = orders.OrderCreate(accountID = account_id, data=data)

    client.request(r)


In [ ]:
def create_market_order_with_trailing_sl_only(instrument, order_type, order_quantity, trailing_sl_pip):
    order_quantity = order_quantity  if order_type == "BUY" else -abs(order_quantity)
    trailingStopLossOnFill = TrailingStopLossDetails(distance=trailing_sl_pip/10000)
    data = {
        "order": {
            "units": str(order_quantity),
            "instrument": instrument,
            "timeInForce": "FOK",
            "type": "MARKET",
            "positionFill": "DEFAULT",
            "trailingStopLossOnFill": trailingStopLossOnFill.data,
        },
    }
    client = oandapyV20.API(access_token=access_token, environment = "practice" or "live")
    r = orders.OrderCreate(accountID = account_id, data=data)

    client.request(r)

In [ ]:
def create_limit_order(instrument, order_type, order_quantity, order_price):
    order_quantity = order_quantity  if order_type == "BUY" else -abs(order_quantity)
    data = {
      "order": {
        "price" : str(order_price),
        "units": str(order_quantity),
        "instrument": instrument,
        "timeInForce": "GTC",
        "type": "LIMIT",
        "positionFill": "DEFAULT"
      }
    }
    client = oandapyV20.API(access_token=access_token, environment = "practice" or "live")
    r = orders.OrderCreate(accountID = account_id, data=data)

    client.request(r)


In [ ]:
def create_limit_order_with_sl_tp(instrument, order_type, order_quantity,order_price, sl_price, tp_price):
    order_quantity = order_quantity  if order_type == "BUY" else -abs(order_quantity)
    data = {
        "order": {
            "price" : str(order_price),
            "units": str(order_quantity),
            "instrument": instrument,
            "timeInForce": "GTC",
            "type": "LIMIT",
            "positionFill": "DEFAULT",
            "stopLossOnFill": {
                "price": str(sl_price)
            },
            "takeProfitOnFill": {
                "price": str(tp_price)
            }
        },
    }
    client = oandapyV20.API(access_token=access_token, environment = "practice" or "live")
    r = orders.OrderCreate(accountID = account_id, data=data)

    client.request(r)



In [ ]:
def create_limit_order_with_sl_only(instrument, order_type, order_quantity,order_price, sl_price):
    order_quantity = order_quantity  if order_type == "BUY" else -abs(order_quantity)
    data = {
        "order": {
            "price" : str(order_price),
            "units": str(order_quantity),
            "instrument": instrument,
            "timeInForce": "GTC",
            "type": "LIMIT",
            "positionFill": "DEFAULT",
            "stopLossOnFill": {
                "price": str(sl_price)
            },

        },
    }
    client = oandapyV20.API(access_token=access_token, environment = "practice" or "live")
    r = orders.OrderCreate(accountID = account_id, data=data)

    client.request(r)



In [ ]:
create_market_order_with_trailing_sl_tp("EUR_USD", "BUY", 100, 5, 1.1950)